In [1]:
from pyspark.sql import functions as F

POLICY_COUNT = 120000          # adjust as needed
START_DATE   = "2014-01-01"
END_DATE     = "2024-12-31"

POLICIES_OUT_PATH = "Files/source/policies_parquet"


StatementMeta(, f84d60f9-4799-468d-88f1-e79bc4c996d1, 3, Finished, Available, Finished)

In [2]:
customers_src_path = "Files/source/customers_csv"
agents_src_path    = "Files/source/agents_csv"

customers = (
    spark.read.option("header", "true")
    .csv(customers_src_path)
    .select("customer_id", "join_date", "risk_segment", "preferred_agent_id")
)

agents = (
    spark.read.option("header", "true")
    .csv(agents_src_path)
    .select("agent_id", "region", "channel")
)

display(customers)
display(agents)

StatementMeta(, f84d60f9-4799-468d-88f1-e79bc4c996d1, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c03d34bc-788a-4845-98a2-da5f3eab1d33)

SynapseWidget(Synapse.DataFrame, 611b5d99-c60a-4f4e-ac23-4be05e81636f)

In [4]:
from pyspark.sql.window import Window

# Create an indexed customers table
cust_indexed = (
    customers.select("customer_id", "preferred_agent_id")
    .withColumn("cust_idx", F.row_number().over(Window.orderBy(F.monotonically_increasing_id())))
)

cust_max = cust_indexed.agg(F.max("cust_idx").alias("max_idx")).collect()[0]["max_idx"]

policies_base = (
    spark.range(POLICY_COUNT)
    .withColumnRenamed("id", "pol_num")
    .withColumn("policy_id", F.format_string("P%09d", F.col("pol_num") + F.lit(1)))
    .withColumn("cust_idx", (F.floor(F.rand() * F.lit(cust_max)) + 1).cast("int"))
)

policies_joined = (
    policies_base.join(cust_indexed, on="cust_idx", how="left")
    .drop("cust_idx")
)

# Add policy attributes
policies_final = (
    policies_joined
    .withColumn("policy_type", F.element_at(F.array([F.lit(x) for x in policy_types]), (F.floor(F.rand() * len(policy_types)) + 1).cast("int")))
    .withColumn("plan_name",   F.element_at(F.array([F.lit(x) for x in plan_names]),   (F.floor(F.rand() * len(plan_names)) + 1).cast("int")))
    .withColumn("premium_frequency", F.element_at(F.array([F.lit(x) for x in freqs]), (F.floor(F.rand() * len(freqs)) + 1).cast("int")))
    .withColumn("start_date", F.to_date(F.expr(f"date_add('{START_DATE}', cast(rand() * datediff('{END_DATE}','{START_DATE}') as int))")))
    .withColumn("term_years", (F.floor(F.rand() * 5) + 1).cast("int"))
    .withColumn("end_date", F.to_date(F.expr("add_months(start_date, term_years * 12)")))
    .withColumn(
        "policy_status",
        F.when(F.col("end_date") < F.current_date(), F.lit("Expired"))
         .when(F.rand() < 0.06, F.lit("Cancelled"))
         .otherwise(F.lit("Active"))
    )
    # Sum insured varies by type
    .withColumn(
        "sum_insured",
        F.when(F.col("policy_type") == "Auto",   (F.floor(F.rand() * 40000) + 10000).cast("int"))
         .when(F.col("policy_type") == "Health", (F.floor(F.rand() * 150000) + 50000).cast("int"))
         .when(F.col("policy_type") == "Home",   (F.floor(F.rand() * 450000) + 150000).cast("int"))
         .otherwise(                              (F.floor(F.rand() * 800000) + 200000).cast("int"))  # Life
    )
    .withColumn("deductible", (F.floor(F.rand() * 4500) + 500).cast("int"))
    .withColumnRenamed("preferred_agent_id", "agent_id")
    .drop("pol_num")
)

display(policies_final)

StatementMeta(, f84d60f9-4799-468d-88f1-e79bc4c996d1, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cb33b98a-12cd-49c6-8e45-0de1cd9f739d)

In [5]:
(policies_final
 .repartition(8)
 .write.mode("overwrite")
 .parquet(POLICIES_OUT_PATH))

print(f"✅ Policies Parquet written to: {POLICIES_OUT_PATH}")

StatementMeta(, f84d60f9-4799-468d-88f1-e79bc4c996d1, 7, Finished, Available, Finished)

✅ Policies Parquet written to: Files/source/policies_parquet


In [6]:
print("Policy count:", policies_final.count())
display(policies_final.groupBy("policy_type", "policy_status").count().orderBy("policy_type","policy_status"))

StatementMeta(, f84d60f9-4799-468d-88f1-e79bc4c996d1, 8, Finished, Available, Finished)

Policy count: 120000


SynapseWidget(Synapse.DataFrame, 9ccc2b18-783c-4745-af17-f161231764c1)